This noteboook is designed as a prototype script for grabbing articles from the JSON file.

First let's create a small batch of the JSON objects.

In [15]:
pathway = "/Users/marci/Desktop/RC_2015-01"

In [1]:
import json
import shelve
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stopwords.append('[deleted]')
#stopwords.append('deleted')

In [2]:
print stopwords

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u

## Function to quickly make a python shelve database

In [16]:
def json_to_db(db_name,num_of_comments):
    

    '''
    Goes through the txt file, grabs the JSON objects in it.
    Uses shelve to create a database, with key= Subreddit and continually appending text to value.
    '''
    # open database
    d = shelve.open(db_name)
    
    # Set counter
    count = 0
    
    # Iterate through N JSON objects in file
    with open(pathway) as myfile:
        for item in myfile:
            if count < num_of_comments:
                
                # Load the JSON object
                json_object = json.loads(item)
                
                # Now append additional text to database
                if d.has_key(json_object['subreddit']):
                    
                    temp = d[json_object['subreddit']]      # extracts the copy
                    temp = temp+' '+json_object['body']     # mutates the copy
                    d[json_object['subreddit']] = temp      # stores the copy right back, to persist it

                # Or set the first entry for that subreddit
                else:
                    d[json_object['subreddit']] = json_object['body']
                
    
                count += 1
            else:
                d.close()
                break

In [17]:
json_to_db('10000_comments',10)

In [18]:
d = shelve.open('10000_comments.db')

In [20]:
d['needadvice']

u"This is a great question, and I want to thank you for asking it. However, I don't have any answers. I'm interested in learning more myself. "

## Tokenization and Cleaning Text

Here we focus on converting the large string into a list of words (tokenized and stopwords removed)

In [3]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

# Output tokenizes text and removes any stopwords and then outptus lowercased words
output = [word.lower() for word in tokenizer.tokenize(d['needadvice']) if not word.lower() in stopwords]

NameError: name 'd' is not defined

In [23]:
len(tokenizer.tokenize(d['needadvice']))

28

In [24]:
len(output)

10

## Final word cleaner code is below

In [4]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

stopwords = stopwords.words('english')
stopwords.append('[deleted]')
tokenizer = RegexpTokenizer(r'\w+')

def text_cleaner(text):
    '''
    INPUT: string of body text
    OUTPUT: List of tokenized lower case words with stopwords removed
    '''
    
    
    # Output tokenizes text and removes any stopwords and then outptus lowercased words
    return [word.lower() for word in tokenizer.tokenize(text) if not word.lower() in stopwords]
    
#text_cleaner(d['MakeupAddiction'])

# MONGO DB Setup

This focuses on setting up a MongoDB and Push the new body of comments to the collection

In [18]:
import pymongo

# Connect with db
#reddit_data = pymongo.Connection()['reddit_database']['comments']

def update_mongo(collection,subreddit,comment):
    '''
    INPUT: subreddit name and comment as a cleaned list of words
    RESULT: Updates MongoDB reddit_data database with $push
    '''
    
    # NOTE: This pushes the entire list, so final result is a list of comment lists
    # Use itertools.chain.from_iterable(reddit_data[subreddit]) to combine this for doc2vec
    collection.update({'subreddit': subreddit},
                      {'$push':{'body':comment}},upsert = True,
                      safe=True)


In [181]:
comment_test = text_cleaner(d['MakeupAddiction'])
update_mongo(reddit_data,'MakeupAddiction',comment_test)

## Clears database

In [19]:
def clear_mongo(mongodb):
    '''
    INPUT: A mongodb database
    RESULT: Based on user input, clears database.
    '''
    ans = raw_input('WARNING: THIS WILL CLEAR THE DATABASE. ARE YOU SURE? y/n')
    if ans == 'y':
        mongodb.remove()
        print 'Database has been cleared.'
    else:
        print 'Database not cleared.'
        
        
#clear_mongo(reddit_data)

## Take in SubReddit, return document list of words

In [5]:
import itertools 

def get_doc_list(db_collection,subreddit):
    '''
    INPUT: Subreddit name
    OUTPUT: A document = list of words for that subreddit
    '''
    
    for obj in db_collection.find({'subreddit':subreddit}):
        return list(itertools.chain.from_iterable(obj['body']))

In [29]:
#get_doc_list(reddit_data,'AskReddit')

## Create a MongoDB 

In [7]:
import pymongo

def create_or_connect(database,collection):
    '''
    INPUT: Database name (str) and collection (name)
    OUTPUT: Either creates the database or connects to it if it already exists.
            Returns the collections in the database cursor.
    '''
    
    return pymongo.Connection()[database][collection]

#create_or_connect('check','test_coll') works!

## Create MongoDB from my Reddit Data

In [22]:
# Clear the db first
clear_mongo(reddit_data)

NameError: name 'reddit_data' is not defined

In [8]:
# Make or Connect to Database
reddit_data = create_or_connect('reddit_database','reddit_comments')

def reddit_data_pusher(db_name,num_of_comments):
    
    '''
    Goes through the txt file, grabs the JSON objects in it.
    Uses MongoDB to create a database, with key= Subreddit and continually appending clean text word list to value.
    '''
    
    # Set counter
    count = 0
    
    # Iterate through N JSON objects in file
    with open(pathway) as myfile:
        for i in xrange(10000100):
            myfile.next()
        for item in myfile:
            if count < num_of_comments:
                
                # Load the JSON object
                json_object = json.loads(item)
                
                # Clean and tokenize text
                body = text_cleaner(json_object['body'])
                
                if str(json_object['subreddit']) != 'AskReddit':
                    
                    try:
                
                        # $push to MongoDB
                        update_mongo(reddit_data,json_object['subreddit'],body)

                        # For safety puposes
                        count += 1
                    except:
                        print 'error in pushing to mongodb'
                    finally:
                        pass
                else:
                    pass
            else:
                break

## Create a MongoDB with N comments

In [ ]:
#clear_mongo(reddit_data)
#reddit_data = create_or_connect('reddit_database','reddit_comments')
#reddit_data_pusher(reddit_data,5000000)

## Begin Doc2Vec Model Training

Let's try to train a model on a very small corpus (10,000 comments)

In [9]:
def sentence_gen(database):
    for item in database.find():
        words=list(itertools.chain.from_iterable(item['body']))
        yield LabeledSentence(words,[str(item['subreddit'])])

In [10]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import LabeledSentence
import gensim.models.doc2vec
import multiprocessing

cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1

In [11]:
# Doc2Vec(dbow,d100,n5,mc2,t8)
d2v_reddit_model = Doc2Vec( dm=0, size=100, negative=5, hs=0, min_count=2, workers=cores)
d2v_reddit_model.build_vocab(sentence_gen(reddit_data))
#d2v_reddit_model.train(sentence_gen(reddit_data))

for epoch in range(10):
    d2v_reddit_model.train(sentence_gen(reddit_data))
    d2v_reddit_model.alpha -= 0.002  # decrease the learning rate
    d2v_reddit_model.min_alpha = d2v_reddit_model.alpha  # fix the learning rate, no decay

In [12]:
d2v_reddit_model.save('weekend_5mil_reddit_model')

## Create Generator of Labeled Documents

In [ ]:
for epoch in range(10):
    d2v_reddit_model.train(sentence_gen(reddit_data))
    d2v_reddit_model.alpha -= 0.002  # decrease the learning rate
    d2v_reddit_model.min_alpha = d2v_reddit_model.alpha  # fix the learning rate, no decay

In [2]:
qq = LabeledSentence(words=['alpha','beta','charlie'],labels=['first'])

In [256]:
model = Doc2Vec(alpha=0.025, min_alpha=0.025)  # use fixed learning rate
model.build_vocab(sentence_gen(reddit_data))
model.train(sentence_gen(reddit_data))
''' 
for epoch in range(10):
    model.train(sentence_gen(reddit_data))
    model.alpha -= 0.002  # decrease the learning rate
    model.min_alpha = model.alpha  # fix the learning rate, no decay
'''

' \nfor epoch in range(10):\n    model.train(sentence_gen(reddit_data))\n    model.alpha -= 0.002  # decrease the learning rate\n    model.min_alpha = model.alpha  # fix the learning rate, no decay\n'

In [51]:
test_load = Doc2Vec.load('10mil_reddit_model')

In [60]:
test_load.docvecs.most_similar(positive=['gaming'])

[('AskGames', 0.7069275379180908),
 ('bloodborne', 0.7028169631958008),
 ('mirrorsedge', 0.6918472051620483),
 ('Fallout4', 0.6884211301803589),
 ('JRPG', 0.6809813976287842),
 ('yakuzagames', 0.6753157377243042),
 ('persona4golden', 0.6734381914138794),
 ('stronghold', 0.6616673469543457),
 ('ChivalryGame', 0.6592487096786499),
 ('CitiesSkylines', 0.6581976413726807)]

In [42]:
d2v_reddit_model.most_similar('AskReddit')

[('Offensive_Wallpapers', 0.8445112705230713),
 ('Waxpen', 0.8392848372459412),
 ('FlossCrew', 0.8378405570983887),
 ('HereComesTheBoom', 0.8365437984466553),
 ('SEO', 0.8363415002822876),
 ('Blondeass', 0.8363173604011536),
 ('Starmade', 0.8361822366714478),
 ('aal', 0.8359136581420898),
 ('bulletjournal', 0.8358200788497925),
 ('Iowa', 0.8352646231651306)]

In [45]:
d2v_reddit_model.most_similar('guns')

[('guncontrol', 0.8933992981910706),
 ('TankPorn', 0.8930748105049133),
 ('joerogan2', 0.8922608494758606),
 ('omnisregion', 0.8921858072280884),
 ('GearsOfWar', 0.8897333145141602),
 ('ajlee', 0.889672040939331),
 ('GirlsPlayingSports', 0.8894907832145691),
 ('sex_comics', 0.8894057273864746),
 ('lgg2', 0.8887702822685242),
 ('japancirclejerk', 0.8885037302970886)]

In [47]:
d2v_reddit_model.most_similar('guns')

[('guncontrol', 0.9355584979057312),
 ('TankPorn', 0.9350743889808655),
 ('GirlsPlayingSports', 0.9337582588195801),
 ('Innie', 0.9314041137695312),
 ('howyoudoin', 0.9310590624809265),
 ('AnimalsBeingBros', 0.9301647543907166),
 ('shittynosleep', 0.9299746155738831),
 ('FIU', 0.9298853874206543),
 ('pornID', 0.9298760890960693),
 ('VirginiaTech', 0.9296820163726807)]

In [19]:
d2v_reddit_model.docvecs.most_similar(positive=['Python'])

[('javascript', 0.8189147710800171),
 ('brainfuck', 0.8057856559753418),
 ('algotrading', 0.8013641834259033),
 ('programming', 0.7919216156005859),
 ('ocaml', 0.7918417453765869),
 ('AskProgramming', 0.7833486795425415),
 ('shittyprogramming', 0.7772594690322876),
 ('ruby', 0.7738404273986816),
 ('beginnerprojects', 0.7691885828971863),
 ('Clojure', 0.7660512924194336)]

NEED TO DO:

PUT THIS INTO A CLEAN OOP PY FILE
GET SPARK ROLLING!

In [1]:
from gensim.models.doc2vec import LabeledSentence

In [62]:
d2v_reddit_model.docvecs.most_similar('Python')

[('learnpython', 0.7819709181785583),
 ('PHPhelp', 0.756676197052002),
 ('lolphp', 0.7540650367736816),
 ('gngr', 0.753192663192749),
 ('learnjavascript', 0.7517881989479065),
 ('learnjava', 0.749946653842926),
 ('node', 0.7479515075683594),
 ('dotnet', 0.7461504936218262),
 ('Clojure', 0.7455533742904663),
 ('MSAccess', 0.734258770942688)]

In [20]:
test = Doc2Vec.load('weekend_5mil_reddit_model')

MONGO DB --- 359 comments per second....
    

In [33]:
test.docvecs.most_similar(positive=['California'])

[('sanfrancisco', 0.7710226774215698),
 ('bears', 0.7244086265563965),
 ('trueshreddit', 0.7229745388031006),
 ('randpaul', 0.7196967601776123),
 ('upstate_new_york', 0.7077169418334961),
 ('MAguns', 0.7043018937110901),
 ('texas', 0.7027081847190857),
 ('newdetroitstyle', 0.7024891376495361),
 ('321', 0.7024003267288208),
 ('Colorado', 0.7016831636428833)]